In [10]:
from bs4 import BeautifulSoup

In [34]:
import os
import sys
import urlextract
import re

def remove_urls(text):
    url_extractor = urlextract.URLExtract()
    urls = list(set(url_extractor.find_urls(text)))
    urls.sort(key=lambda url: len(url), reverse=True)
    for url in urls:
        text = text.replace(url, " URL ")
    return text

def remove_nums(text):
    text = re.sub(r'\d+(?:\.\d*(?:[eE]\d+))?', 'NUMBER', text)
    return text
def remove_punctuation(text):
    text = re.sub(r'\W+', ' ', text, flags=re.M)
    return text

url_extractor = urlextract.URLExtract()
path = './rsc/spam'

for file in os.listdir(path):
    current = os.path.join(path, file)
    soap = BeautifulSoup(open(current, "rb"))
    text = soap.get_text()
    text = remove_urls(text)
    text = remove_nums(text)
    text = remove_punctuation(text)
    text = text.lower()
    print(text)
    break

AttributeError: 'str' object has no attribute 'walk'